**Sprint 2: Data Wrangling**

In [19]:
#Dependencies
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
print(os.listdir("../data"))

['site_dic.pkl', 'sample_submission.csv', 'test_sessions.csv', 'train_sessions.csv']


**Load the dataset**

In [20]:
PATH_TO_DATA = ('../data')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

**Basic Preprocessing**

Convert timestamps into pd.datetime

In [21]:
#list columns for easy access
sites_cols = ['site%s' % i for i in range(1, 11)]
times_cols = ['time%s' % i for i in range(1,11)]

In [22]:
#convert timestamps to pd.datetime
train_df[times_cols] = train_df[times_cols].apply(pd.to_datetime)
test_df[times_cols] = test_df[times_cols].apply(pd.to_datetime)

In [23]:
train_df = train_df.sort_values(by = 'time1')
test_df = test_df.sort_values(by = 'time1')

In [24]:
test_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
65540,21,2014-05-01 17:14:03,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
64199,23,2014-05-02 07:52:08,66.0,2014-05-02 07:54:08,63.0,2014-05-02 07:54:08,2626.0,2014-05-02 07:55:09,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
2268,979,2014-05-02 07:57:51,73.0,2014-05-02 07:59:34,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT
29734,66,2014-05-02 08:05:16,69.0,2014-05-02 08:05:17,67.0,2014-05-02 08:05:17,70.0,2014-05-02 08:05:17,71.0,2014-05-02 08:05:17,68.0,2014-05-02 08:05:17,71.0,2014-05-02 08:05:18,70.0,2014-05-02 08:05:18,69.0,2014-05-02 08:05:18,67.0,2014-05-02 08:05:18
77048,167,2014-05-02 08:05:32,167.0,2014-05-02 08:05:33,359.0,2014-05-02 08:05:34,167.0,2014-05-02 08:05:34,167.0,2014-05-02 08:05:35,305.0,2014-05-02 08:09:19,306.0,2014-05-02 08:09:20,306.0,2014-05-02 08:09:22,979.0,2014-05-02 08:09:54,68.0,2014-05-02 08:12:46


**Feature Engineering**

In [25]:
class DataPrep(BaseEstimator, TransformerMixin):
    """
        Fill NaN with zero values.
    """
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        sites = ['site%s' % i for i in range(1, 11)]
        return X[sites].fillna(0).astype('int')

We combine the site id to a single string for the CountVectorizer.

In [26]:
class TokenizerPrep(BaseEstimator, TransformerMixin):
    """
        Prepare site URLs for tokenizer (CountVectorizer).
    """
    def fit(self, X, y = None):
        return self
    def transform(self, X, y = None):
        X = X.values.tolist()
        # Each row is combine into 1 string, with each value (siteID) seperated by 1 whitespace.
        # Put these strings into 1 list and return it.
        return [" ".join([str(site) for site in row]) for row in X]

Some features explored in the EDA showed significant differences between Alice and Intruder. Let's put them into the dataset.

In [27]:
class FeatureAdder(BaseEstimator, TransformerMixin):
    """
        Extracts and engineers session-based and temporal features from the input DataFrame.
        Features added:
        minutes -- Duration of the session in minutes.
        start_month -- Encoded year and month of session start (YYYYMM).
        year -- Year of session start.
        month -- Month of session start.
        start_week -- Encoded year and ISO week of session start (YYYYWW).
        start_day -- Day of year of session start.
        start_hour -- Hour of session start.
        dow -- Day of week of session start (0=Monday).
        is_weekend -- 1 if session starts on weekend, else 0.
        work_hours -- 1 if session starts during typical work hours (9 a.m. to 5 p.m.) and during Alice's active days (Monday, Tuesday, Thursday and Friday), else 0.
        avg_time_per_site -- Average time spent per unique site in the session (minutes).
        unique_sites -- Number of unique sites visited in the session.
        repeated_site_visits -- Number of repeated site visits in the session.
        ----------------------------------------------------------------------
        Returns:
            np.ndarray: Array of engineered features (no original columns).
    """
    def fit(self, X, y = None):
        return self
    def transform(self, X, y = None):
        # Get the min and max of the timestamps.
        min_times = X[times_cols].min(axis=1)
        max_times = X[times_cols].max(axis=1)
        # Duration of the session in minutes
        minutes = ((max_times - min_times).dt.total_seconds() / 60).round(2)
        # Start month of the session
        start_month = min_times.dt.year * 100 + min_times.dt.month
        # Year of the session (Ordinal)
        year_map = {2013: 0, 2014: 1, 2015: 2}
        year = min_times.dt.year.map(year_map)
        # Month of the session
        month = min_times.dt.month
        # Start week of the session
        start_week = min_times.dt.year * 100 + min_times.dt.isocalendar().week
        # Start day of the session
        start_day = min_times.dt.dayofyear
        # Start hour of the session
        start_hour = min_times.dt.hour
        # Day of week of the session
        dow = min_times.dt.weekday
        # Whether of not the session started on the weekend.
        is_weekend = min_times.dt.weekday.isin([5, 6]).astype(int)
        # Whether of not the session started on an active Alice's day and work hour.
        work_hours = (
            min_times.dt.weekday.isin([0, 1, 3, 4]) & #Monday, Tuesday, Thursday and Friday
            (min_times.dt.hour >= 9) & (min_times.dt.hour <= 17) #9 a.m. to 5 p.m.
        ).astype(int)

        # Average time spent per site in 1 session
        site_counts = X[sites_cols].nunique(axis=1) # Total number of sites 
        avg_time_per_site = minutes.div(site_counts.where(site_counts > 0, np.nan)).fillna(0) # Minute duration / number of sites (0 if site_counts = 0)
        
        # Number of unique sites visited in 1 session
        unique_sites = X[sites_cols].nunique(axis=1).astype(int)

        # Number of repeated site visits in 1 session
        repeated_site_visits = len(sites_cols) - X[sites_cols].nunique(axis=1).astype(int)

        # Scalling the larger features (YYYYMM format) to be within [0,1] interval.
        scaled = MinMaxScaler().fit_transform(
            np.c_[
                start_month.values,
                start_week.values
            ]
        )
        start_month_scaled = scaled[:, 0]
        start_week_scaled = scaled[:, 1]

        # Return as numpy array (no DataFrame, no original columns)
        return np.c_[
            minutes.values,
            start_month_scaled,
            year.values,
            month.values,
            start_week_scaled,
            start_day.values,
            start_hour.values,
            dow.values,
            is_weekend.values,
            work_hours.values,
            avg_time_per_site.values,
            unique_sites.values,
            repeated_site_visits.values
        ]

We make 2 pipelines:
* vectorizer_pipeline: prepares the dataset for tokenizer by imputing NaNs in siteID columns, and combining them into a list of strings. CountVectorizer() converts a collection of text into a matrix of token counts. ngram_range of (1,3) means that the vectorizer will extract unigrams (single site IDs), bigrams (pairs of site IDs) and trigrams (triplets of site IDs). Only the 10000 most frequent n-grams are kept as features. Returns a a 2D-array of these features.\
For example, given array = ['00' , '01', '00', '02'], the CountVectorizer will return a matrix that counts up the frequency of each element (token): \
[['00', '01', '02'], \
[2, 1, 1]] 
* feature_pipeline: Returns a 2D-array of engineered features from the given dataset.

In [28]:
#Initialize the pipelines
vectorizer_pipeline = Pipeline([('tokenizer_prep', TokenizerPrep()), ('tokenizer', CountVectorizer(ngram_range=(1,4), max_features=10000))])

feature_pipeline = Pipeline([('feature_engineering', FeatureAdder())])

FeatureUnion performs the transformation processes in parallel, concatenating them together at the end.

In [29]:
#Preprocessing pipeline
preprocessing = FeatureUnion(transformer_list=[('vectorizer', vectorizer_pipeline), ('feature_engineering', feature_pipeline)])

In [30]:
X_train = preprocessing.fit_transform(train_df)
X_test = preprocessing.transform(test_df)

y_train = train_df["target"].astype('int').values

In [31]:
# The transformed dataset is very large. We sample a few rows to take a look.
sample = X_train[:5].toarray()
print("NaNs in sample:", np.isnan(sample).sum())
print("Infs in sample:", np.isinf(sample).sum())
print("Sample rows:\n", sample)

# Check sparsity
print("Nonzero elements:", X_train.nnz)
print("Sparsity: {:.2f}%".format(100 * X_train.nnz / (X_train.shape[0] * X_train.shape[1])))

# Check min/max (Make sure everything is scaled)
print("Sample min:", sample.min())
print("Sample max:", sample.max())

NaNs in sample: 0
Infs in sample: 0
Sample rows:
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.00000000e+00 8.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.48850000e+01
  2.00000000e+00 8.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.16666667e-02
  6.00000000e+00 4.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e-02
  5.00000000e+00 5.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 5.00000000e-03
  6.00000000e+00 4.00000000e+00]]
Nonzero elements: 13865139
Sparsity: 0.55%
Sample min: 0.0
Sample max: 29.77


The dataset looks good, without NaNs, infs or large outliers, but is too large to be manually inspected. Let's run a simple model to see if the preprocessing steps done have improved the model.

In [32]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from xgboost import XGBClassifier

In [33]:
time_split = TimeSeriesSplit(n_splits=10)

my_model_1 = XGBClassifier()

In [34]:
cv_scores = cross_val_score(my_model_1, X_train, y_train, cv=time_split, 
                        scoring='roc_auc', n_jobs=3)
print(cv_scores.mean())

0.888619299099707


The cv_score is greater than 0.5 for a logistic regression problem, which suggest that the added features improved the model.

This will be the baseline for other models, which we will explore in the next phase.